In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from warnings import WarningMessage

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('./competetion_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./competetion_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./competetion_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./competetion_data/t10k-images-idx3-ubyte.gz
Extracting ./competetion_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

Log_dir = "./competetion_data/"

In [4]:
_input = tf.placeholder(dtype=tf.float32, shape=[None, time_steps, element_size], name = 'input')
y = tf.placeholder(dtype=tf.float32, shape = [None,num_classes], name = 'labels')

In [5]:
batch_x, batch_y = mnist.train.next_batch(batch_size)

In [6]:
batch_x = batch_x.reshape([batch_size, time_steps, element_size])

In [7]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [8]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope('W_x'):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

In [9]:
def rnn_step(previous_hiddle_state, x):
    current_hidden_state = tf.tanh(
    tf.matmul(previous_hiddle_state, Wh) +
    tf.matmul(x, Wx) + b_rnn)
    
    return current_hidden_state

In [10]:
processed_input = tf.transpose(_input, perm=[1,0,2])
initial_hidden = tf.zeros([batch_size, hidden_layer_size])
all_hidden_states = tf.scan(rnn_step,
                            processed_input,
                            initializer=initial_hidden,
                            name = 'states')

In [13]:
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size,
                                             num_classes],
                                            mean = 0, stddev = .01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                            mean=0, stddev=.01))
        variable_summaries(bl)

def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

with tf.name_scope('linear_layer_weight') as scope:
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

In [14]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=output, labels = y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32))) * 100
    tf.summary.scalar('accuracy', accuracy)
    
merged = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [21]:
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(Log_dir + '/train',
                                        graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(Log_dir + '/test',
                                       graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, time_steps, element_size))
        summary, _ = sess.run([merged, train_step],
                             feed_dict = {_input:batch_x, y:batch_y})
        train_writer.add_summary(summary,i)
        
        if i % 1000 == 0:
            acc, loss = sess.run([accuracy, cross_entropy],
                                feed_dict = {_input:batch_x, y:batch_y})
        print("Iter " + str(i) + ", Miniatch Loss = " + "{:.6f}".format(loss) + ", Training Accuracy = " + 
             "{:.5f}".format(acc))
        
        if i % 10:
            summary, acc = sess.run([merged, accuracy],
                                   feed_dict={_input: test_data,
                                             y: test_label})
            test_writer.add_summary(summary, i)
        test_acc = sess.run(accuracy, feed_dict={_input: test_data,
                                                y: test_label})
        print("Test Accuracy: ", test_acc)

Iter 0, Miniatch Loss = 2.302634, Training Accuracy = 7.03125
Test Accuracy:  9.375
Iter 1, Miniatch Loss = 2.302634, Training Accuracy = 7.03125
Test Accuracy:  9.375
Iter 2, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 3, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 4, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 5, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 6, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 7, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 8, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 9, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 10, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  9.375
Iter 11, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accurac

Iter 98, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  11.71875
Iter 99, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  11.71875
Iter 100, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  9.375
Iter 101, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  9.375
Iter 102, Miniatch Loss = 2.302634, Training Accuracy = 9.37500
Test Accuracy:  7.8125
Iter 103, Miniatch Loss = 2.302634, Training Accuracy = 7.81250
Test Accuracy:  7.8125
Iter 104, Miniatch Loss = 2.302634, Training Accuracy = 7.81250
Test Accuracy:  11.71875
Iter 105, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  11.71875
Iter 106, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  11.71875
Iter 107, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  11.71875
Iter 108, Miniatch Loss = 2.302634, Training Accuracy = 11.71875
Test Accuracy:  11.71875
Iter 109, Miniatch Loss = 

Test Accuracy:  17.1875
Iter 194, Miniatch Loss = 2.302634, Training Accuracy = 17.18750
Test Accuracy:  14.84375
Iter 195, Miniatch Loss = 2.302634, Training Accuracy = 14.84375
Test Accuracy:  14.84375
Iter 196, Miniatch Loss = 2.302634, Training Accuracy = 14.84375
Test Accuracy:  18.75
Iter 197, Miniatch Loss = 2.302634, Training Accuracy = 18.75000
Test Accuracy:  16.40625
Iter 198, Miniatch Loss = 2.302634, Training Accuracy = 16.40625
Test Accuracy:  17.1875
Iter 199, Miniatch Loss = 2.302634, Training Accuracy = 17.18750
Test Accuracy:  25.78125
Iter 200, Miniatch Loss = 2.302634, Training Accuracy = 25.78125
Test Accuracy:  29.6875
Iter 201, Miniatch Loss = 2.302634, Training Accuracy = 25.78125
Test Accuracy:  17.1875
Iter 202, Miniatch Loss = 2.302634, Training Accuracy = 17.18750
Test Accuracy:  32.03125
Iter 203, Miniatch Loss = 2.302634, Training Accuracy = 32.03125
Test Accuracy:  15.625
Iter 204, Miniatch Loss = 2.302634, Training Accuracy = 15.62500
Test Accuracy:  11.

Test Accuracy:  40.625
Iter 290, Miniatch Loss = 2.302634, Training Accuracy = 40.62500
Test Accuracy:  35.15625
Iter 291, Miniatch Loss = 2.302634, Training Accuracy = 40.62500
Test Accuracy:  34.375
Iter 292, Miniatch Loss = 2.302634, Training Accuracy = 34.37500
Test Accuracy:  31.25
Iter 293, Miniatch Loss = 2.302634, Training Accuracy = 31.25000
Test Accuracy:  18.75
Iter 294, Miniatch Loss = 2.302634, Training Accuracy = 18.75000
Test Accuracy:  38.28125
Iter 295, Miniatch Loss = 2.302634, Training Accuracy = 38.28125
Test Accuracy:  35.9375
Iter 296, Miniatch Loss = 2.302634, Training Accuracy = 35.93750
Test Accuracy:  40.625
Iter 297, Miniatch Loss = 2.302634, Training Accuracy = 40.62500
Test Accuracy:  36.71875
Iter 298, Miniatch Loss = 2.302634, Training Accuracy = 36.71875
Test Accuracy:  36.71875
Iter 299, Miniatch Loss = 2.302634, Training Accuracy = 36.71875
Test Accuracy:  19.53125
Iter 300, Miniatch Loss = 2.302634, Training Accuracy = 19.53125
Test Accuracy:  37.5
It

Test Accuracy:  37.5
Iter 386, Miniatch Loss = 2.302634, Training Accuracy = 37.50000
Test Accuracy:  44.53125
Iter 387, Miniatch Loss = 2.302634, Training Accuracy = 44.53125
Test Accuracy:  41.40625
Iter 388, Miniatch Loss = 2.302634, Training Accuracy = 41.40625
Test Accuracy:  37.5
Iter 389, Miniatch Loss = 2.302634, Training Accuracy = 37.50000
Test Accuracy:  28.90625
Iter 390, Miniatch Loss = 2.302634, Training Accuracy = 28.90625
Test Accuracy:  37.5
Iter 391, Miniatch Loss = 2.302634, Training Accuracy = 28.90625
Test Accuracy:  41.40625
Iter 392, Miniatch Loss = 2.302634, Training Accuracy = 41.40625
Test Accuracy:  44.53125
Iter 393, Miniatch Loss = 2.302634, Training Accuracy = 44.53125
Test Accuracy:  40.625
Iter 394, Miniatch Loss = 2.302634, Training Accuracy = 40.62500
Test Accuracy:  42.96875
Iter 395, Miniatch Loss = 2.302634, Training Accuracy = 42.96875
Test Accuracy:  42.96875
Iter 396, Miniatch Loss = 2.302634, Training Accuracy = 42.96875
Test Accuracy:  42.1875


...

Test Accuracy:  96.875
Iter 9570, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.875
Iter 9571, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  98.4375
Iter 9572, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9573, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  98.4375
Iter 9574, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9575, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  97.65625
Iter 9576, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  97.65625
Iter 9577, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  98.4375
Iter 9578, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9579, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  96.875
Iter 9580, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Acc

Iter 9663, Miniatch Loss = 0.022392, Training Accuracy = 95.31250
Test Accuracy:  98.4375
Iter 9664, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  98.4375
Iter 9665, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  96.875
Iter 9666, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.09375
Iter 9667, Miniatch Loss = 0.022392, Training Accuracy = 96.09375
Test Accuracy:  96.875
Iter 9668, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  98.4375
Iter 9669, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  98.4375
Iter 9670, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9671, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  98.4375
Iter 9672, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  96.875
Iter 9673, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  99.21875
Iter 9674,

Test Accuracy:  99.21875
Iter 9756, Miniatch Loss = 0.022392, Training Accuracy = 99.21875
Test Accuracy:  99.21875
Iter 9757, Miniatch Loss = 0.022392, Training Accuracy = 99.21875
Test Accuracy:  99.21875
Iter 9758, Miniatch Loss = 0.022392, Training Accuracy = 99.21875
Test Accuracy:  98.4375
Iter 9759, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  98.4375
Iter 9760, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9761, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9762, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  98.4375
Iter 9763, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  99.21875
Iter 9764, Miniatch Loss = 0.022392, Training Accuracy = 99.21875
Test Accuracy:  95.3125
Iter 9765, Miniatch Loss = 0.022392, Training Accuracy = 95.31250
Test Accuracy:  95.3125
Iter 9766, Miniatch Loss = 0.022392, Training Accuracy = 95.31250
Test

Test Accuracy:  96.875
Iter 9849, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.875
Iter 9850, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.875
Iter 9851, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.875
Iter 9852, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  97.65625
Iter 9853, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  96.09375
Iter 9854, Miniatch Loss = 0.022392, Training Accuracy = 96.09375
Test Accuracy:  96.09375
Iter 9855, Miniatch Loss = 0.022392, Training Accuracy = 96.09375
Test Accuracy:  97.65625
Iter 9856, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  96.875
Iter 9857, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.875
Iter 9858, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  96.875
Iter 9859, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy

Iter 9941, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  97.65625
Iter 9942, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  96.875
Iter 9943, Miniatch Loss = 0.022392, Training Accuracy = 96.87500
Test Accuracy:  95.3125
Iter 9944, Miniatch Loss = 0.022392, Training Accuracy = 95.31250
Test Accuracy:  97.65625
Iter 9945, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  97.65625
Iter 9946, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  97.65625
Iter 9947, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  98.4375
Iter 9948, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  98.4375
Iter 9949, Miniatch Loss = 0.022392, Training Accuracy = 98.43750
Test Accuracy:  97.65625
Iter 9950, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  96.875
Iter 9951, Miniatch Loss = 0.022392, Training Accuracy = 97.65625
Test Accuracy:  98.4375
Iter 99